<a href="https://colab.research.google.com/github/bemakerorg/AIoT_Book_II_RF/blob/main/AIoT_RF_Book_ES_17b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importiamo la libreria TensrFlow e verifichiamo che sia la versione 2.18.0
import tensorflow as tf
print("Current TensorFlow version:", {tf.__version__})
if tf.__version__ != "2.18.0":
    print(f"Current TensorFlow version: {tf.__version__}, switching to 2.18.0")

    # Disistalla la libreria se diversa dalla 2.18.0
    !pip uninstall -y tensorflow

    # Installa la libreria TensorFlow 2.18
    !pip install tensorflow==2.18

    # Dopo l'istallazione occorre riavviare il runtime
    print("TensorFlow 2.18 installed.")
    print("Please click on the Runtime > Restart session and run all.")
else:
    print("TensorFlow 2.18 is already installed.")

In [ ]:
# 1) Import delle librerie necessarie
import numpy as np                                       # NumPy per gli array e operazioni numeriche
from tensorflow import keras                             # Keras API per costruire modelli di reti
import pathlib                                           # pathlib per operazioni su percorsi di file

In [ ]:
# 2) Costruzione del modello Sequenziale con un singolo neurone
my_layer = keras.layers.Dense(units=1, input_shape=[1]) # Crea un layer Dense con 1 neurone e input scalare
model = keras.Sequential([my_layer])                     # Avvolge il layer in un modello Sequential

In [ ]:
# 3) Compilazione del modello
model.compile(
    optimizer='sgd',                                   # Ottimizzatore Stochastic Gradient Descent
    loss='mean_squared_error'                          # Funzione di perdita MSE per regressione
)

In [ ]:
# 4) Preparazione dei dati di training
xs = np.array([4.0, 2.0, -1.0, 1.0, 0.0, 3.0], dtype=float)   # Array degli input
ys = np.array([7.0, 3.0, -3.0, 1.0, -1.0, 5.0], dtype=float)  # Array dei target corrispondenti

In [ ]:
# 5) Addestramento del modello
model.fit(
    xs,                                              # Inputs
    ys,                                              # Target outputs
    epochs=500                                       # Numero di epoche di training
)

In [ ]:
# 6) Stampa del peso W e del bias b appresi
print("Peso e bias appreso:", my_layer.get_weights())  # Restituisce [w, b]

In [ ]:
# 7) Predizione di esempio con x=10.0
print("Predizione con x=10.0:", model.predict(np.array([10.0])))

In [ ]:
# 8) Esportazione del modello in formato SavedModel
saved_dir = 'saved_model/1'                     # Directory di destinazione
model.export(saved_dir)                         # Esporta il modello in SavedModel compatibile TF
print(f"Modello esportato in formato SavedModel in: {pathlib.Path(saved_dir).resolve()}")

In [ ]:
# 9) Configurazione del converter TFLite per il SavedModel
converter = tf.lite.TFLiteConverter.from_saved_model(saved_dir)  # Crea il converter
converter.experimental_enable_resource_variables = True          # Fold delle variabili in costanti
tflite_model = converter.convert()                              # Esegue la conversione

In [ ]:
# 10) Salvataggio del flatbuffer TFLite su disco
tflite_path = pathlib.Path('model_from_savedmodel.tflite')      # Percorso output .tflite
tflite_path.write_bytes(tflite_model)                           # Scrive i byte del modello
print(f"TFLite model salvato in: {tflite_path.resolve()}")

In [ ]:
# 11) Inizializzazione dell’interprete TFLite
interpreter = tf.lite.Interpreter(model_path=str(tflite_path))  # Carica il modello TFLite
interpreter.allocate_tensors()                                  # Alloca memoria per tensori

In [ ]:
# 12) Recupero dei dettagli di input e output
input_details  = interpreter.get_input_details()  # Metadati sul tensore di input
output_details = interpreter.get_output_details() # Metadati sul tensore di output
print("Input details:", input_details)
print("Output details:", output_details)

In [ ]:
# 13) Esecuzione dell’inferenza TFLite
to_predict = np.array([[10.0]], dtype=np.float32)             # Dati di input formattati
interpreter.set_tensor(input_details[0]['index'], to_predict) # Carica il tensore di input
interpreter.invoke()                                         # Esegue il modello
tflite_results = interpreter.get_tensor(output_details[0]['index'])  # Estrae il risultato
print("Risultato TFLite per x=10.0:", tflite_results)        # Stampa l’output dell'inferenza